In [111]:
using FFTW, Test

In [38]:
x = rand(10)
FFTW.r2r!(x, FFTW.REDFT10)

# x = rand(8, 8, 4)
# y = FFTW.r2r!(x, FFTW.REDFT10, 3)

10-element Array{Float64,1}:
  9.319280363453597  
  0.15478431340772214
  2.9336211880336114 
 -0.2881143313943969 
  0.5877363618947404 
  1.1495086839516115 
 -2.2778284990563806 
 -1.5766797189382675 
 -1.147384309562554  
  0.493700605286072  

In [110]:
t = collect(1:16)
cat(t[1:2:N], t[N:-2:2]; dims=1)

16-element Array{Int64,1}:
  1
  3
  5
  7
  9
 11
 13
 15
 16
 14
 12
 10
  8
  6
  4
  2

In [131]:
function dct_1d_gpu!(f)
    N = size(f)[1]
    f .= cat(f[1:2:N], f[N:-2:2]; dims=1)
    FFTW.fft!(f)
    f .*= 2 * exp.(collect(-1im*π*(0:N-1) / (2*N)))
    nothing
end

dct_1d_gpu! (generic function with 1 method)

In [132]:
N = 16
x = Complex.(rand(Float64, N))
y = copy(x)

dct_1d_gpu!(x)

FFTW.r2r!(y, FFTW.REDFT10);

@test real.(x) ≈ real.(y)

Test Passed

In [151]:
Nx, Ny, Nz = 4, 4, 4
t = reshape(collect(1:Nx*Ny*Nz), Nx, Ny, Nz)
cat(t[:, :, 1:2:Nz], t[:, :, Nz:-2:2]; dims=3)

4×4×4 Array{Int64,3}:
[:, :, 1] =
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

[:, :, 2] =
 33  37  41  45
 34  38  42  46
 35  39  43  47
 36  40  44  48

[:, :, 3] =
 49  53  57  61
 50  54  58  62
 51  55  59  63
 52  56  60  64

[:, :, 4] =
 17  21  25  29
 18  22  26  30
 19  23  27  31
 20  24  28  32

In [153]:
factors = 2 * exp.(collect(-1im*π*(0:Nz-1) / (2*Nz)))
repeat(reshape(factors, 1, 1, Nz), Nx, Ny, 1)

4×4×4 Array{Complex{Float64},3}:
[:, :, 1] =
 2.0-0.0im  2.0-0.0im  2.0-0.0im  2.0-0.0im
 2.0-0.0im  2.0-0.0im  2.0-0.0im  2.0-0.0im
 2.0-0.0im  2.0-0.0im  2.0-0.0im  2.0-0.0im
 2.0-0.0im  2.0-0.0im  2.0-0.0im  2.0-0.0im

[:, :, 2] =
 1.84776-0.765367im  1.84776-0.765367im  …  1.84776-0.765367im
 1.84776-0.765367im  1.84776-0.765367im     1.84776-0.765367im
 1.84776-0.765367im  1.84776-0.765367im     1.84776-0.765367im
 1.84776-0.765367im  1.84776-0.765367im     1.84776-0.765367im

[:, :, 3] =
 1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im
 1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im
 1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im
 1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im  1.41421-1.41421im

[:, :, 4] =
 0.765367-1.84776im  0.765367-1.84776im  …  0.765367-1.84776im
 0.765367-1.84776im  0.765367-1.84776im     0.765367-1.84776im
 0.765367-1.84776im  0.765367-1.84776im     0.765367-1.847

In [164]:
function dct_dim3_gpu!(f)
    Nx, Ny, Nz = size(f)
    f .= cat(f[:, :, 1:2:Nz], f[:, :, Nz:-2:2]; dims=3)
    FFTW.fft!(f, 3)

    factors = 2 * exp.(collect(-1im*π*(0:Nz-1) / (2*Nz)))
    f .*= repeat(reshape(factors, 1, 1, Nz), Nx, Ny, 1)
    
    nothing
end

dct_dim3_gpu! (generic function with 1 method)

In [169]:
Nx, Ny, Nz = 4, 4, 4
# x = Complex{Float64}.(reshape(collect(1:Nx*Ny*Nz), Nx, Ny, Nz))
x = Complex.(rand(Float64, Nx, Ny, Nz))
y = copy(x)

dct_dim3_gpu!(x)

FFTW.r2r!(y, FFTW.REDFT10, 3);

@test real.(x) ≈ real.(y)

Test Passed